<a href="https://colab.research.google.com/github/shiki-fujin/DS-MLE-exam/blob/main/text_classification_own_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialization

In [2]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [3]:
! pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 100.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 21.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
! pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 6.8 MB/s eta 0:00:00


In [5]:
# For hyperparam search
! pip install ray[tune]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 46.1 MB/s eta 0:00:00


In [6]:
import random
import pandas as pd
import numpy as np
from IPython.display import display, HTML

In [7]:
from huggingface_hub import notebook_login
import transformers
import datasets
import evaluate
from ray import tune

print(transformers.__version__)

4.25.1


In [8]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


## Fine-tuning using the sample dataset provided : **CoLa**

> Metrics to beat : 
1.  Training Loss	= 0.553600 (reduce)
2.  Validation Loss = 0.594802 (reduce)
3.  Matthews Correlation = 0.455038 (increase)


Prepare dataset

In [9]:
def encode_cola(examples):
    return tokenizer(examples['sentence'], truncation=True)


def load_data(task):
  dataset = datasets.load_dataset('glue', task)
  metric = evaluate.load('glue', actual_task)
  
  encoded_dataset = dataset.map(encode_cola, batched=True)
  train_dataset = encoded_dataset['train'].shard(index=1, num_shards=10)
  eval_dataset = encoded_dataset['validation'].shard(index=1, num_shards=10)
  return train_dataset, eval_dataset, metric


actual_task = 'cola'
pretrained_model_name = 'distilbert-base-uncased'
tokenizer = transformers.AutoTokenizer.from_pretrained(pretrained_model_name, use_fast=True)
train_dataset, eval_dataset, metric = load_data(actual_task)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

...

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/cola/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Define Training

In [10]:
# seed for reproducibility
random_seed = 30

In [11]:
# Define metrics for Trainer
def compute_metrics(eval_pred): 
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)


def model_init():
    return transformers.AutoModelForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=2)

# trigger downloading and caching
transformers.AutoModelForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=2)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [12]:
output_name = f"{pretrained_model_name}-finetuned-{actual_task}-01"
metric_name = 'matthews_correlation'
training_args = transformers.TrainingArguments(
    output_name,
    seed = random_seed,
    evaluation_strategy = "epoch",
    metric_for_best_model=metric_name)

trainer = transformers.Trainer(
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset, 
    eval_dataset=eval_dataset, 
    model_init=model_init,
    compute_metrics=compute_metrics)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1

Hyperparameter search

In [13]:
from ray.tune.schedulers import ASHAScheduler

In [15]:
tune_config = {
    'per_device_train_batch_size': 16,
    'per_device_eval_batch_size': 16,
    'num_train_epochs': tune.choice([2, 5]),
    'learning_rate' : tune.choice([2e-5, 3e-5, 5e-5]),
    'weight_decay' : tune.choice([0, 0.1, 0.01, 0.05, 0.001])}


best_run = trainer.hyperparameter_search(hp_space=lambda _: tune_config, n_trials=5, direction='maximize', scheduler=ASHAScheduler(metric="objective", mode="max"))

No `resources_per_trial` arg was passed into `hyperparameter_search`. Setting it to a default value of 1 CPU for each trial.


== Status ==
Current time: 2023-01-24 03:13:14 (running for 00:00:01.16)
Memory usage on this node: 4.3/12.7 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: None
Resources requested: 1.0/2 CPUs, 0/0 GPUs, 0.0/7.27 GiB heap, 0.0/3.64 GiB objects
Result logdir: /root/ray_results/_objective_2023-01-24_03-13-12
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+-------------------+-----------------+--------------------+----------------+
| Trial name             | status   | loc               |   learning_rate |   num_train_epochs |   weight_decay |
|------------------------+----------+-------------------+-----------------+--------------------+----------------|
| _objective_096be_00000 | RUNNING  | 172.28.0.12:11351 |           3e-05 |                  5 |           0    |
| _objective_096be_00001 | PENDING  |                   |           5e-05 |                  2 |           0    |
| _

(_objective pid=11351) Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight']
(_objective pid=11351) - This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=11351) - This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=11351) Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base

== Status ==
Current time: 2023-01-24 03:13:21 (running for 00:00:08.97)
Memory usage on this node: 4.6/12.7 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: None
Resources requested: 2.0/2 CPUs, 0/0 GPUs, 0.0/7.27 GiB heap, 0.0/3.64 GiB objects
Result logdir: /root/ray_results/_objective_2023-01-24_03-13-12
Number of trials: 5/5 (3 PENDING, 2 RUNNING)
+------------------------+----------+-------------------+-----------------+--------------------+----------------+
| Trial name             | status   | loc               |   learning_rate |   num_train_epochs |   weight_decay |
|------------------------+----------+-------------------+-----------------+--------------------+----------------|
| _objective_096be_00000 | RUNNING  | 172.28.0.12:11351 |           3e-05 |                  5 |           0    |
| _objective_096be_00001 | RUNNING  | 172.28.0.12:11452 |           5e-05 |                  2 |           0    |
| _

  0%|          | 1/270 [00:05<26:28,  5.91s/it]
(_objective pid=11452) Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight']
(_objective pid=11452) - This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=11452) - This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=11452) Some weights of DistilBertForSequenceClassification were not initiali

...

== Status ==
Current time: 2023-01-24 03:54:34 (running for 00:41:21.40)
Memory usage on this node: 5.9/12.7 GiB 
Using AsyncHyperBand: num_stopped=1
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: 0.39831375340784575 | Iter 1.000: 0.39745930922967404
Resources requested: 1.0/2 CPUs, 0/0 GPUs, 0.0/7.27 GiB heap, 0.0/3.64 GiB objects
Result logdir: /root/ray_results/_objective_2023-01-24_03-13-12
Number of trials: 5/5 (1 RUNNING, 4 TERMINATED)
+------------------------+------------+-------------------+-----------------+--------------------+----------------+-------------+
| Trial name             | status     | loc               |   learning_rate |   num_train_epochs |   weight_decay |   objective |
|------------------------+------------+-------------------+-----------------+--------------------+----------------+-------------|
| _objective_096be_00004 | RUNNING    | 172.28.0.12:11351 |           3e-05 |                  2 |           0.01 |             |
| _objective_096be_0

(_objective pid=11351) 
  0%|          | 0/7 [00:00<?, ?it/s]
(_objective pid=11351) 
 29%|██▊       | 2/7 [00:01<00:02,  1.92it/s]
(_objective pid=11351) 
 43%|████▎     | 3/7 [00:02<00:03,  1.27it/s]


== Status ==
Current time: 2023-01-24 03:54:39 (running for 00:41:26.41)
Memory usage on this node: 5.9/12.7 GiB 
Using AsyncHyperBand: num_stopped=1
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: 0.39831375340784575 | Iter 1.000: 0.39745930922967404
Resources requested: 1.0/2 CPUs, 0/0 GPUs, 0.0/7.27 GiB heap, 0.0/3.64 GiB objects
Result logdir: /root/ray_results/_objective_2023-01-24_03-13-12
Number of trials: 5/5 (1 RUNNING, 4 TERMINATED)
+------------------------+------------+-------------------+-----------------+--------------------+----------------+-------------+
| Trial name             | status     | loc               |   learning_rate |   num_train_epochs |   weight_decay |   objective |
|------------------------+------------+-------------------+-----------------+--------------------+----------------+-------------|
| _objective_096be_00004 | RUNNING    | 172.28.0.12:11351 |           3e-05 |                  2 |           0.01 |             |
| _objective_096be_0

(_objective pid=11351) 
 57%|█████▋    | 4/7 [00:04<00:03,  1.33s/it]
(_objective pid=11351) 
 71%|███████▏  | 5/7 [00:05<00:02,  1.28s/it]
(_objective pid=11351) 
 86%|████████▌ | 6/7 [00:06<00:01,  1.20s/it]
(_objective pid=11351) 
100%|██████████| 7/7 [00:07<00:00,  1.05s/it]
                                                
 50%|█████     | 54/108 [04:21<04:21,  4.85s/it]
2023-01-24 03:54:42,490	INFO tune.py:762 -- Total run time: 2489.61 seconds (2489.48 seconds for the tuning loop).


== Status ==
Current time: 2023-01-24 03:54:42 (running for 00:41:29.50)
Memory usage on this node: 5.9/12.7 GiB 
Using AsyncHyperBand: num_stopped=2
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: 0.39831375340784575 | Iter 1.000: 0.39221834132626593
Resources requested: 0/2 CPUs, 0/0 GPUs, 0.0/7.27 GiB heap, 0.0/3.64 GiB objects
Result logdir: /root/ray_results/_objective_2023-01-24_03-13-12
Number of trials: 5/5 (5 TERMINATED)
+------------------------+------------+-------------------+-----------------+--------------------+----------------+-------------+
| Trial name             | status     | loc               |   learning_rate |   num_train_epochs |   weight_decay |   objective |
|------------------------+------------+-------------------+-----------------+--------------------+----------------+-------------|
| _objective_096be_00000 | TERMINATED | 172.28.0.12:11351 |           3e-05 |                  5 |           0    |    0.455038 |
| _objective_096be_00001 | TERMIN

In [17]:
# Peek at the best run
best_run

BestRun(run_id='096be_00000', objective=0.45503755920364, hyperparameters={'per_device_train_batch_size': 16, 'per_device_eval_batch_size': 16, 'num_train_epochs': 5, 'learning_rate': 3e-05, 'weight_decay': 0})

In [19]:
# Set optimized-params from best_run
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

# run training again
trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased were not used when in

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,No log,0.582940,0.000000
2,No log,0.507451,0.482108
3,No log,0.575652,0.440386
4,No log,0.890605,0.398314
5,No log,0.874866,0.455038


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 105
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 105
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this messa

TrainOutput(global_step=270, training_loss=0.3094563802083333, metrics={'train_runtime': 1249.6204, 'train_samples_per_second': 3.421, 'train_steps_per_second': 0.216, 'total_flos': 23497439738160.0, 'train_loss': 0.3094563802083333, 'epoch': 5.0})

In [20]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 105
  Batch size = 16


{'eval_loss': 0.8748661875724792,
 'eval_matthews_correlation': 0.45503755920364,
 'eval_runtime': 11.4735,
 'eval_samples_per_second': 9.152,
 'eval_steps_per_second': 0.61,
 'epoch': 5.0}

In [21]:
# Peek at the optimized-params
best_run.hyperparameters.items()

dict_items([('per_device_train_batch_size', 16), ('per_device_eval_batch_size', 16), ('num_train_epochs', 5), ('learning_rate', 3e-05), ('weight_decay', 0)])

In [22]:
print('learning_rate :', trainer.args.learning_rate)
print('num_train_epochs :', trainer.args.num_train_epochs)
print('per_device_train_batch_size :', trainer.args.per_device_train_batch_size)

learning_rate : 3e-05
num_train_epochs : 5
per_device_train_batch_size : 16


In [23]:
# Save results to hub
trainer.push_to_hub()

Cloning https://huggingface.co/naksu/distilbert-base-uncased-finetuned-cola-01 into local empty directory.


Download file pytorch_model.bin:   0%|          | 15.4k/255M [00:00<?, ?B/s]

Download file runs/Jan23_05-04-15_bcf53e1a6248/1674450303.3054147/events.out.tfevents.1674450303.bcf53e1a6248.…

Download file training_args.bin: 100%|##########| 3.37k/3.37k [00:00<?, ?B/s]

Download file runs/Jan23_05-25-04_bcf53e1a6248/1674451520.2122216/events.out.tfevents.1674451520.bcf53e1a6248.…

...

Saving model checkpoint to distilbert-base-uncased-finetuned-cola-01
Configuration saved in distilbert-base-uncased-finetuned-cola-01/config.json
Model weights saved in distilbert-base-uncased-finetuned-cola-01/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-cola-01/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-cola-01/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 32.0k/255M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 3.37k/3.37k [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/naksu/distilbert-base-uncased-finetuned-cola-01
   fbc4c12..dc2ef45  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/naksu/distilbert-base-uncased-finetuned-cola-01
   fbc4c12..dc2ef45  main -> main

To https://huggingface.co/naksu/distilbert-base-uncased-finetuned-cola-01
   dc2ef45..b4666db  main -> main

   dc2ef45..b4666db  main -> main



'https://huggingface.co/naksu/distilbert-base-uncased-finetuned-cola-01/commit/dc2ef456c39a7d7697acc6f1a0ebc9363a864679'